#Install yolov8

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.2/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

#Install Roboflow Supervision

In [ ]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.21.0


#prediction of model

In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO

# Function to downsample video
def downsample_video(input_path, output_path, scale_factor):
    # Open the video file
    cap = cv2.VideoCapture(input_path)

    # Get the video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_width / scale_factor), int(frame_height / scale_factor)))

    # Read until video is completed
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Downsample the frame
            downsized_frame = cv2.resize(frame, (int(frame_width / scale_factor), int(frame_height / scale_factor)))
            # Write the downsized frame to the output video file
            out.write(downsized_frame)
        else:
            break

    # Release the video capture and writer objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Input and output file paths
input_video_path = '/content/drive/MyDrive/Copy of Start to End Hyderabad Part 3 720P.mp4'
output_video_path = '/content/drive/MyDrive/downsampled_video.mp4'

# Define the downsampling factor
downsampling_factor = 2  # Adjust this according to your needs

# Call the function to downsample the video
downsample_video(input_video_path, output_video_path, downsampling_factor)

print("Video downsampling complete.")

# Load YOLO model
model = YOLO('/content/drive/MyDrive/Road_furniture_weights/content/runs/detect/train/weights/best.pt')

# Define callback function to be used in video processing
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # Model prediction on single frame and conversion to supervision Detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    # Annotate the frame with all detected classes
    labels = [f"{model.model.names[class_id]} {confidence:0.2f}"
              for confidence, class_id in zip(detections.confidence, detections.class_id)]

    bbox_annotator = sv.BoundingBoxAnnotator(thickness=3)
    label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=2)  # Adjust text_thickness and text_scale to reduce font size

    annotated_frame = bbox_annotator.annotate(scene=frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    return annotated_frame

# Process the whole video
SOURCE_VIDEO_PATH = '/content/drive/MyDrive/downsampled_video.mp4'
TARGET_VIDEO_PATH = '/content/drive/MyDrive/Prediction_Start to End Hyderabad Part 3_road_furniture.mp4'

sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=TARGET_VIDEO_PATH,
    callback=callback
)

print("Video processing complete.")


Video downsampling complete.
